<a href="https://colab.research.google.com/github/shkim0116/Lecture-Recommendation-NLP/blob/main/preprocess_d2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm

data = pd.read_csv('/content/drive/Shareddrives/NLP모델링/코드 연습/문서/reviewtokenized.csv')
data['강의평토큰화'] = [list(map(str, data['강의평토큰화'].iloc[i][2:-2].split("', '"))) for i in range(len(data))]
data['target'] = [list(map(float, data['target'].iloc[i][1:-1].split(', '))) for i in range(len(data))]

In [ ]:
data.head()

,Unnamed: 0,강의명/교수명,강의평,target,강의평토큰화
0,0,대학기본영어Ⅱ/오진숙,어렵지 않은 내용으로 탄탄한 기본기를 다져주는 유익한 수업,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]","[어렵다, 않다, 내용, 탄탄하다, 기, 본기, 다지다, 유익하다, 수업]"
1,1,대학기본영어Ⅱ/오진숙,교수님 너무 귀여우심 사랑해요자잘한 과제들 귀찮긴 했지만 시간투자 그렇게 오래 걸리...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]","[교수, 너무, 귀, 여우, 심, 사랑, 하다, 자잘하다, 과제, 귀찮다, 하다, ..."
2,2,대학기본영어Ⅱ/오진숙,완전 좋음 학기 때 내가 살아있던 이유 기말 때 갑자기 시험을 세번 본 거는 당황스...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]","[완전, 좋다, 학기, 때, 내, 살다, 이유, 기, 말, 때, 갑자기, 시험, 세..."
3,3,대학기본영어Ⅱ/오진숙,영어 말하는 데에 자신감을 얻고 싶으면 추천 다만 과제는 자잘한게 많고 필자는 학기...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]","[영어, 말, 하다, 데, 자신감, 얻다, 싶다, 추천, 다만, 과제, 자잘하다, ..."
4,4,대학기본영어Ⅱ/오진숙,수강당시에는 조모임 비율도 높았고 발음 중시하시는건 왜 그러시는지 잘 이해가지 않지...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]","[수강, 당시, 조, 모임, 비율, 높다, 발음, 중시, 하다, 왜, 그렇다, 자다..."


In [ ]:
uniquecla = list(set(data['강의명/교수명'].values))

In [ ]:
aggword = []
for j in tqdm(uniquecla):
    index = data[data['강의명/교수명']==j].index
    word = ''
    for i in index:
        word += data['강의평'][i] + '. '
    aggword.append(word[:-1])

100%|██████████| 386/386 [00:01<00:00, 371.59it/s]


In [ ]:
target = []
for i in uniquecla:
    target.append(data['target'][data['강의명/교수명']==i].values[0])

In [ ]:
data1 = pd.DataFrame({'강의명/교수명' : uniquecla,\
                      'target' : target,\
                      '강의평' : aggword})

In [ ]:
data1

,강의명/교수명,target,강의평
0,INTRODUCTION TO MANAGEMENT/천은영,"[0.5, 1.0, 0.0, 0.5, 0.0, 0.5]",피할 수 있으면 피하세요 아 맞다 근데 이거 이지 ㅋㅋㅋㅋㅋ. -이 교수님 솔직히 ...
1,생명공학수학/김영식,"[1.0, 0.5, 0.0, 0.5, 0.5, 0.5]",솔직히 강의력은 학년때 들었던 미적벡 교수님들이 훨씬 비교도 안되게 좋음 하지만 학...
2,영어음성학/이석재,"[1.0, 0.5, 1.0, 1.0, 0.5, 0.5]",교수님이 정말 좋으시고 학생들을 은근히 위해주시는게 보입니다 학점도 후하시고 설명도...
3,언론학개론/윤영철,"[1.0, 0.0, 0.0, 0.5, 0.5, 0.5]",대체 어디를 가르치시는건지 수업내내 궁금함 강의노트는 왜있는거고 있으면 왜따라가지않...
4,사회복지개론/채주석,"[1.0, 0.0, 0.5, 1.0, 0.5, 0.5]",빛수업은 개론 수업이다보니 지루할 수 있으나 교수님이 너어무 좋습니다 배려도 많이 ...
...,...,...,...
381,RC자기주도활동(2)/한봉환,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",비대면이라 아쉬웠지만 여러 가지 경험해보면서 즐거운 시간 보냈습니다. 막판에 몰아서...
382,YONSEI RC 101-생명공학과/홍혜경,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",과제는 거의 없고 틀어놓기만 하면 되는 수업입니다 저는 틀어놓고 협곡에서 시간을 보...
383,유기화학/심태보,"[0.5, 0.5, 0.0, 0.0, 0.0, 0.5]",만 피합시다 왜 녹강을 실시간으로 틀어주는 지도 모르겠고 그렇다고 교수님이 실시간 ...
384,한문(1)/김기완,"[1.0, 1.0, 0.5, 1.0, 0.0, 0.5]",학점을 정말 퍼주십니다 정말정말 게으르신 사람이 아니라면 노베도 무조건 받아간다고 ...


In [ ]:
aaa = data1[['강의명/교수명', '강의평']]
texttag_1 = [(aaa.iloc[i][1], [aaa.iloc[i][0],]) for i in range(len(aaa))]
TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in texttag_1]

In [ ]:
model = Doc2Vec(TRAIN_documents, vector_size = 100, window = 3, min_count = 5, workers = 4)

In [ ]:
inputvec = []
for i in uniquecla:
    inputvec.append(list(model.docvecs[i]))

In [ ]:
data1['input'] = inputvec
data1 = data1[['강의명/교수명','input','target']]

In [ ]:
data1.to_csv('/content/drive/Shareddrives/NLP모델링/코드 연습/문서/d2vnormver.csv')